Librerías necesarias:
```
pip install cognee kuzu
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

```

In [6]:
import requests
import dlt

## 1. Versión de dlt

In [ ]:

!dlt --version

dlt 1.13.0


## 2. dlt pipelines

In [7]:
# fuente
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc



Ahora necesitamos definir el destino.
Donde la data será almacenada en el archivo db.qdrant 

In [8]:
from dlt.destinations import qdrant

qdrant_destinantion = qdrant(
    qd_path = "d.qdrant"
)
# Data will be stored in db.qdrant file placed in current working directory.

In [9]:
# Crear el pipeline
pipeline = dlt.pipeline(
    pipeline_name= "zoomcamp_pipeline",
    destination= qdrant_destinantion, 
    dataset_name= "zoomcamp_tagged_data"

)

load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  8.84it/s]


Run started at 2025-07-10 22:16:09.616602+00:00 and COMPLETED in 10.24 seconds with 4 steps.
Step extract COMPLETED in 0.49 seconds.

Load package 1752185775.168173 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- _dlt_pipeline_state: 1 row(s)
- zoomcamp_data: 948 row(s)

Load package 1752185775.168173 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.12 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.11 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/workshops/d.qdrant location to store data
Load package 1752185775.168173 is LOADED and contains no failed jobs

Step run COMPLETED in 10.24 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.11 seconds
1 load package(s) were loaded to destinatio

Las filas cargadas son 948

## ¿Qué modelo es usado?

In [11]:
import json

with open("d.qdrant/meta.json", "r") as file:
    data = json.load(file)

print(data)

{'collections': {'zoomcamp_tagged_data': {'vectors': {'fast-bge-small-en': {'size': 384, 'distance': 'Cosine', 'hnsw_config': None, 'quantization_config': None, 'on_disk': None, 'datatype': None, 'multivector_config': None}}, 'shard_number': None, 'sharding_method': None, 'replication_factor': None, 'write_consistency_factor': None, 'on_disk_payload': None, 'hnsw_config': None, 'wal_config': None, 'optimizers_config': None, 'init_from': None, 'quantization_config': None, 'sparse_vectors': None, 'strict_mode_config': None}, 'zoomcamp_tagged_data__dlt_version': {'vectors': {'fast-bge-small-en': {'size': 384, 'distance': 'Cosine', 'hnsw_config': None, 'quantization_config': None, 'on_disk': None, 'datatype': None, 'multivector_config': None}}, 'shard_number': None, 'sharding_method': None, 'replication_factor': None, 'write_consistency_factor': None, 'on_disk_payload': None, 'hnsw_config': None, 'wal_config': None, 'optimizers_config': None, 'init_from': None, 'quantization_config': None,

In [14]:
data['collections']['zoomcamp_tagged_data__dlt_version']

{'vectors': {'fast-bge-small-en': {'size': 384,
   'distance': 'Cosine',
   'hnsw_config': None,
   'quantization_config': None,
   'on_disk': None,
   'datatype': None,
   'multivector_config': None}},
 'shard_number': None,
 'sharding_method': None,
 'replication_factor': None,
 'write_consistency_factor': None,
 'on_disk_payload': None,
 'hnsw_config': None,
 'wal_config': None,
 'optimizers_config': None,
 'init_from': None,
 'quantization_config': None,
 'sparse_vectors': None,
 'strict_mode_config': None}